In [17]:
import numpy as np
import pulp
import pandas as pd
import warnings

warnings.filterwarnings('ignore')

# Load the data from the CSV file
file_path = '../data/food_nutrition.csv'
nutrition_data = pd.read_csv(file_path)

# Display the data to understand its structure and contents
food_names = nutrition_data['name'].tolist()
nutrition_data.drop(['unit'], axis=1, inplace=True)       # drop the `unit` column
nutrition_data = nutrition_data.reset_index(drop=True)

# Separate the nutrition data for the foods and the lower/upper bounds
food_nutrition = nutrition_data.iloc[2:]
lower_bounds = nutrition_data.iloc[0, 1:]
upper_bounds = nutrition_data.iloc[1, 1:]

# Protein的上下限与WEIGHT有关 => WEIGHT * [.8, 2]
# Fat的上下限与energy有关 => energy * [0.2, 0.35]
WEIGHT = 60             # 60kg body weight
lower_bounds[2] *= WEIGHT
upper_bounds[2] *= WEIGHT
lower_bounds[3] = lower_bounds[1] * 0.2
upper_bounds[3] = upper_bounds[1] * 0.35


YEAR = 30       # 游戏时长（秒）
# 示例数据
# F: 食物中的营养成分矩阵
# L: 营养成分的最小需求
# U: 营养成分的最大限制
# 假设 F 是食物和营养成分的矩阵，L 和 U 分别是营养成分的下界和上界
F = food_nutrition.drop(['name'], axis=1).to_numpy() * 365 * 6  # 每份食物大概600g

SCALAR1, SCALAR2 = 0.75, 1.1    # 用来调整上下界
L = lower_bounds.to_numpy() * SCALAR1 * YEAR * 365  
U = upper_bounds.to_numpy() * SCALAR2 * YEAR * 365


# 创建线性规划问题
lp_prob = pulp.LpProblem("Nutrition_Optimization", pulp.LpMinimize)

# 创建变量
n_foods = F.shape[0]
food_vars = [pulp.LpVariable(f'food_{food_names[i+2]}', lowBound=0, upBound=30, cat='Integer') for i in range(n_foods)]

# 目标函数：最小化食物数量
lp_prob += pulp.lpSum(food_vars)

# 添加营养成分的约束
for j in range(F.shape[1]):
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) >= L[j]
    lp_prob += pulp.lpSum(F[i][j] * food_vars[i] for i in range(n_foods)) <= U[j]

# 求解问题
lp_prob.solve(pulp.PULP_CBC_CMD(msg=0))


# 打印结果
count, non_zero = 0, 0
if pulp.LpStatus[lp_prob.status] == 'Optimal':
    print("找到最优解")
    for var in food_vars:
        count += int(var.varValue)
        non_zero += (1 if var.varValue > 0 else 0)
        print("'" + var.name[5:] + "': " + str(int(var.varValue)) + ",")
        # print(int(var.varValue), "个", var.name)
else:
    print("无法找到最优解")
print('Total food count: ', count, ' Non zero count: ', non_zero)

找到最优解
'watermelon': 0,
'apple': 0,
'banana': 1,
'avocado': 0,
'broccoli': 2,
'pink_salmon': 0,
'chicken': 1,
'beef': 2,
'arugula': 2,
'bread': 0,
'egg': 9,
'corn': 2,
'beer': 1,
'vodka': 0,
'coffee': 1,
'noodles': 0,
'rice': 1,
'milk': 0,
'yogurt': 0,
'tofu': 2,
'muffin': 3,
'corn_oil': 4,
'mango': 0,
'cilantro': 0,
'sugar': 0,
'soy_milk': 0,
'carrot': 16,
'pumpkin': 0,
'potato': 0,
Total food count:  47  Non zero count:  14
